# Data Preparation

## About Data
Video data from across the web and self-recorded videos is used to create the input dataset. 
Training data consists of 6 different poses as follows:
- Padamasan (Lotus Pose)
- Bhujangasana (Snake Pose)
- Shavasana (Corpse Pose) 
- Tadasana (Mountain Pose)
- Trikonasana (Triangle Pose)
- Vrikshasana (Tree Pose)

## Processing
The video data is preprocessed to create time series data consisting of angles at the body joints frame by frame while performing the yoga. The video input is passed through Mediapipe's Blazepose to get the coordinates(33 points on the body) of which the coordinates of wrist,  elbow, shoulder,  hip,  knee,  ankle  for both right and left are used to calculate 8 angle(including left and right both side) for the following groups: 
- Wrist, Elbow, Shoulder 
- Elbow, Shoulder, Hip 
- Shoulder, Hip, Knee 
- Hip, Knee, Ankle 

The angles are calculated for 30 equidistant data points through out the user pose.


In [54]:
import cv2
import mediapipe as mp
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
# from fastdtw import fastdtw
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

lst1 = []
singleList = []

def calculateAngle(x):
    '''
    Given the coordinated(x,y) calculate the angel formed by them
    
    Parameters:
        x(List[(x,y)]) : List of 3 pairs of x and y coordinates 
    
    Returns:
        angle(float): Angle made by the 3 coordinates(c1, c2, c3) at c2
    '''
    a = np.array(x[0])
    b = np.array(x[1])
    c = np.array(x[2])
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle>180.0:
        angle = 360-angle
    return angle

def calculate_angles(path):
        '''
    Given the path of the video calculate angle time series during the pose
    
    Parameters:
        path : Path to the video file
    
    Returns:
        df(Pandas.dataframe):Dataframe consisting of 8 angles for each frame
    '''
    cap = cv2.VideoCapture(path)
    lst = []
    
    angledict = {'leftWES':[], 'leftESH':[],'leftSHK':[], 'leftHKA':[], 'rightWES':[], 'rightESH':[], 'rightSHK':[],'rightHKA':[]}
    with mp_pose.Pose(
        smooth_landmarks=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
        while cap.isOpened() :
            success, image = cap.read()
            if not success:
                break
            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.    
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             frame_height, frame_width = image.shape[:2]
            
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            # image.flags.writeable = False
            results = pose.process(image)
            
            # Declaring aliases
            landmarks = results.pose_landmarks.landmark
            setOfJoints = mp_pose.PoseLandmark
            
            # Storing the coordinates as a pair of x and y values
            leftWrist = [landmarks[setOfJoints.LEFT_WRIST].x, landmarks[setOfJoints.LEFT_WRIST].y]
            leftElbow = [landmarks[setOfJoints.LEFT_ELBOW].x, landmarks[setOfJoints.LEFT_ELBOW].y]
            leftShoulder = [landmarks[setOfJoints.LEFT_SHOULDER].x, landmarks[setOfJoints.LEFT_SHOULDER].y]
            leftHip = [landmarks[setOfJoints.LEFT_HIP].x, landmarks[setOfJoints.LEFT_HIP].y]
            leftKnee = [landmarks[setOfJoints.LEFT_KNEE].x, landmarks[setOfJoints.LEFT_KNEE].y]
            leftAnkle = [landmarks[setOfJoints.LEFT_ANKLE].x, landmarks[setOfJoints.LEFT_ANKLE].y]
            rightWrist = [landmarks[setOfJoints.RIGHT_WRIST].x, landmarks[setOfJoints.RIGHT_WRIST].y]
            rightElbow = [landmarks[setOfJoints.RIGHT_ELBOW].x, landmarks[setOfJoints.RIGHT_ELBOW].y]
            rightShoulder = [landmarks[setOfJoints.RIGHT_SHOULDER].x, landmarks[setOfJoints.RIGHT_SHOULDER].y]
            rightHip = [landmarks[setOfJoints.RIGHT_HIP].x, landmarks[setOfJoints.RIGHT_HIP].y]
            rightKnee = [landmarks[setOfJoints.RIGHT_KNEE].x, landmarks[setOfJoints.RIGHT_KNEE].y]
            rightAnkle = [landmarks[setOfJoints.RIGHT_ANKLE].x, landmarks[setOfJoints.RIGHT_ANKLE].y]
            
            #  Creating a dictionary to calculate different angles
            angleDict = {'leftWES': (leftWrist, leftElbow, leftShoulder), 
                         'leftESH': (leftElbow, leftShoulder, leftHip), 
                         'leftSHK': (leftShoulder, leftHip, leftKnee), 
                         'leftHKA': (leftHip, leftKnee, leftAnkle), 
                         'rightWES': (rightWrist, rightElbow, rightShoulder), 
                         'rightESH': (rightElbow, rightShoulder, rightHip), 
                         'rightSHK': (rightShoulder, rightHip, rightKnee), 
                         'rightHKA': (rightHip, rightKnee, rightAnkle)}
           
            
            # Processing the angle dict to calculate different angles
            for angle in angleDict:
                x=calculateAngle(angleDict[angle])
                angledict[angle].append(x)

    cap.release()
    cv2.destroyAllWindows()
    df = pd.DataFrame(angledict)
    # print(df.head())
    return df

In [54]:
# Calculating the angle time series 
dfins = calculate_angles('yoga_min.mp4')

# Storing the dataframe to a csv
dfins.to_csv('yoga3.csv', index=False)


## Processed Data

In [20]:
df = pd.read_csv('yoga3.csv')

In [21]:
df

,leftWES,leftESH,leftSHK,leftHKA,rightWES,rightESH,rightSHK,rightHKA
0,172.768991,25.432565,120.074887,31.466746,169.035355,25.560993,115.471929,33.662575
1,172.168818,26.081544,121.832135,30.830461,169.151667,27.481450,113.628079,31.221266
2,170.429261,28.899776,122.535209,30.829244,165.946378,28.784643,114.215600,31.166409
3,168.264156,31.548129,123.391039,31.765528,162.150498,30.724038,114.957871,32.568336
4,166.630244,33.817120,124.282068,34.303610,160.926532,33.989186,114.271365,33.210670
...,...,...,...,...,...,...,...,...
170,167.030613,11.224975,120.366534,24.749314,160.164557,8.136002,116.546473,34.272147
171,167.094827,11.324279,120.997462,25.147070,160.659154,8.203846,116.029822,34.299070
172,166.984537,11.075818,121.176234,24.901472,160.861050,8.490985,115.830933,34.237287
173,166.786962,11.424646,119.918815,23.801803,161.058076,8.380051,116.207768,34.009205


Now 30 equidistant point are selected to train the model.
